In [12]:
import os
import pandas as pd

folder_path = r"F:\vscode\stock_analysis\env\Scripts\segregated_csv"
all_data=[]

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        df= pd.read_csv(os.path.join(folder_path,file))
        symbol=file.replace(".csv","")
        df['Ticker']=symbol
        all_data.append(df)

combined_df = pd.concat(all_data)
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df=combined_df.sort_values(by=['Ticker','date']).reset_index(drop=True)

combined_df['prev_close'] = combined_df.groupby('Ticker')['close'].shift(1)
combined_df['daily_return'] = (combined_df['close'] - combined_df['prev_close']) / combined_df['prev_close']

combined_df.head(10)

,Ticker,close,date,high,low,month,open,volume,prev_close,daily_return
0,ADANIENT,2387.25,2023-10-03 05:30:00,2424.90,2372.00,2023-10,2418.00,2019899,NaN,NaN
1,ADANIENT,2464.95,2023-10-04 05:30:00,2502.75,2392.25,2023-10,2402.20,2857377,2387.25,0.032548
2,ADANIENT,2466.35,2023-10-05 05:30:00,2486.50,2446.40,2023-10,2477.95,1132455,2464.95,0.000568
3,ADANIENT,2478.10,2023-10-06 05:30:00,2514.95,2466.05,2023-10,2466.35,1510035,2466.35,0.004764
4,ADANIENT,2442.60,2023-10-09 05:30:00,2459.70,2411.30,2023-10,2440.00,1408224,2478.10,-0.014325
5,ADANIENT,2498.30,2023-10-10 05:30:00,2517.95,2443.00,2023-10,2443.00,1771910,2442.60,0.022804
6,ADANIENT,2488.60,2023-10-11 05:30:00,2538.00,2482.50,2023-10,2533.90,1627836,2498.30,-0.003883
7,ADANIENT,2506.35,2023-10-12 05:30:00,2521.75,2490.65,2023-10,2499.90,1804818,2488.60,0.007133
8,ADANIENT,2454.55,2023-10-13 05:30:00,2495.00,2422.35,2023-10,2488.90,2278682,2506.35,-0.020668
9,ADANIENT,2429.35,2023-10-16 05:30:00,2464.70,2421.85,2023-10,2454.55,803856,2454.55,-0.010267


In [15]:
path=r"F:\vscode\stock_analysis\env\Scripts"
output_file = os.path.join(path,"combined_stocks.csv")
combined_df.to_csv(output_file,index=False)
print("All stocks are combined")

All stocks are combined


In [8]:
import pandas as pd

merged_df = pd.read_csv("combined_stocks.csv")
sector_df = pd.read_csv("Sector_data - Sheet1.csv")
sector_df['Ticker'] = sector_df['Symbol'].str.extract(r':\s*([A-Z\-\&]+)')
ticker_to_sector={}
for ticker in merged_df['Ticker'].unique():
    match=None
    for _, row in sector_df.iterrows():
        if ticker in row['Symbol']:
            match=row['sector']
            break
        ticker_to_sector[ticker]=match

ticker_to_sector = sector_df.set_index('Ticker')['sector'].to_dict()
merged_df['sector']=merged_df['Ticker'].map(ticker_to_sector)
manual_fixes = {"ADANIENT" : "MISCELLANEOUS",
                "BHARTIARTL" : "TELECOM"}
for ticker, sector in manual_fixes.items():
    merged_df.loc[merged_df['Ticker'] == ticker, 'sector'] = sector
merged_df.to_csv("sectorwise_data.csv", index=False)
print("Sector column added")

Sector column added
